





## 🏢Job Recommendation Sytem





Installing required libraries:

In [ ]:
import pandas as pd
import numpy as np
import nltk
#nltk.download()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
nltk.download('punkt')
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stopword = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
comb_jobs = pd.read_csv('/content/data15Avril.csv')
user = pd.read_csv('/content/jobseekersfinal.csv')
Profil = pd.read_csv('/content/profil2.csv')
CoursData=pd.read_csv('/content/coursfinal.csv')
comb_jobs = comb_jobs.rename(columns={'Unnamed: 0': 'jobID'})
user = user.rename(columns={'Unnamed: 0': 'ApplicantID'})

### JobOffers Dataset
we will merge all the columns in order to create a corpus of text for each job
We will only extract the columns **JobID**, **Clean job title** ,**Skills** for creating the recommendation system

In [ ]:
comb_jobs_df = comb_jobs[['jobID', 'CleanTitle','Skills_Description','Country','workplaceType','jobType']]
comb_jobs_df['Skills_Description'] = comb_jobs_df['Skills_Description'].apply(str).astype(str)
comb_jobs_df['Skills_Description']= comb_jobs_df['Skills_Description'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)
comb_jobs_df['Text'] = comb_jobs_df['CleanTitle'].map(str) + ' ' + comb_jobs_df['Skills_Description'] 


In [ ]:
comb_jobs

### JobSeeker Dataset
we will merge all the columns in order to create a corpus of text for each user
We will only extract the columns **ApplicantID**, **Headline** ,**Skills** for creating the recommendation system

In [ ]:
user 

,ApplicantID,headline,fullName,allSkills
0,0,has studied at the private high school of engi...,Montassar Fenina,"Arduino IDE, AutoCAD, Mechanical Engineering, ..."
1,1,junior data scientist at value,Med Amine Konyali,"Java, PL/SQL,.NET framework, Automatic Learnin..."
2,2,chief industrial project,Oussama Bougaalech,"Quality Management, English, French, Project P..."
3,3,Head of Team R&D for Electric Vehicles in the ...,Taoufik Chaouachi,"SCADA, Automation, Automation control, piping ..."
4,4,full stack developer at private high school of...,Oussama Mhamdi,"React.js, Golang, Firebase, Node.js, React, Mo..."
...,...,...,...,...
788,810,web developer fullstack,Ismail BEN ALI,"redux, AWS Lambda, Microsoft SQL Server, JavaS..."
789,811,full stack developer at sfr business distribution,Achref Hcini,"MySQL, Git, Framework Symfony, Scrum, Vue.js, ..."
790,812,frontend developer @ bable smart cities,Imene Tbini,"React.js, Node.js, JavaScript, PHP, Java, Angu..."
791,813,Teacher in the Spirit (Private School of Engin...,Rym Alouane,"XML, E-business, Web Marketing, SEO, Business ..."


A common column called "text" will now contain all of  the columns in order to find similarities between the job offers and the applicant qualifications

In [ ]:
user['Text'] = user['headline'] + " " + user['allSkills']

### Profile Dataset
A common column called "text" will now contain all of  the columns in order to find similarities between the job offers and Profil

In [ ]:
Profil['Technologies'] = Profil['Technologies'].apply(str).astype(str)
Profil['Technologies']= Profil['Technologies'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)
Profil['Technologies'] = Profil['Technologies'].apply(str).astype(str)
Profil['Technologies']= Profil['Technologies'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)
Profil['Title'] = Profil['Title'].apply(str).astype(str)
Profil['Title']= Profil['Title'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)

In [ ]:
Profil['Text'] = Profil['Technologies'] + " " + Profil['Title']
Profil.head()

,Unnamed: 0,Field,Technologies,Company,Location,Title,Text
0,0,BI,"visualization, sci, artificial intelligence, e...","['Powersolv, Inc.' 'Piper Companies' 'ProofID'...",['États-Unis' 'Tunisie' 'Afrique du Sud' 'Égyp...,Network Engineer / MS Server Support Consultan...,"visualization, sci, artificial intelligence, e..."
1,1,DS,"erwin, machine learning, accounting, multivari...",['Raytheon Intelligence & Space' 'Snapchat' 'A...,"['États-Unis' 'Canada' 'Taïwan' 'Europe, Moyen...",Machine Learning Software Engineer I Hybrid\n ...,"erwin, machine learning, accounting, multivari..."
2,2,Dev,"rest api, machine learning, ruby, web app, jav...",['CareerTeam' 'ROCKEN' 'ERNI' 'Enova' 'Mindera...,['Suisse' 'Maroc' 'États-Unis' 'United Kingdom...,Java Software Engineer Software Engineer Java\...,"rest api, machine learning, ruby, web app, jav..."
3,3,Electrical,"Process engineering, DevOps, Internet of Thing...",['Four Life Sciences' 'KONE' 'Synergie Belgium...,['Netherlands' 'United Kingdom' 'Belgium' 'Ger...,Field Service Engineer Lift Service Engineer S...,"Process engineering, DevOps, Internet of Thing..."
4,4,Embarked,"system testing, deep learning, systems enginee...",['Ford Motor Company' 'Privafy' 'ROCKEN' 'Cons...,['États-Unis' 'Suisse' 'Switzerland' 'Netherla...,iot embedded software engineer embedded rtos s...,"system testing, deep learning, systems enginee..."


#### **Term Ferquency - Inverse Document Frequency**
The code you provided is using the CountVectorizer to convert a collection of text documents into a matrix of token counts 

In [ ]:
count_vect = CountVectorizer()
# Fitting and transforming the vectorizer
count_comb = count_vect.fit_transform((comb_jobs_df['Text'])) #fitting and transforming the vector
count_comb

<3984x3863 sparse matrix of type '<class 'numpy.int64'>'
	with 54738 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vect = TfidfVectorizer()
# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((comb_jobs_df['Text'])) 
tfidf_comb

<3984x3863 sparse matrix of type '<class 'numpy.float64'>'
	with 54738 stored elements in Compressed Sparse Row format>

#**Building the Recommender Systems** 

## Users recommended job dataset
In order to build a collaborative-based job recommendation system, you will need a dataset of users who have recommended jobs. This data will be used to generate recommendations for  users based on their preferences 

In [ ]:
def get_recommendation(top, comb_jobs_all, scores,user):
  recommendation = pd.DataFrame(columns = ['Applicant_ID', 'jobID',  'CleanTitle', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Applicant_ID'] = user
      recommendation.at[count, 'jobID'] = comb_jobs_all['jobID'][i]
      recommendation.at[count, 'CleanTitle'] = comb_jobs_all['CleanTitle'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [ ]:
user = user.drop(user.index[-30:])

**Computing the Cosine Similarity using  countvector**
1.   we used the CountVectorizer  to create a count vector representation of users . This will convert each user Text description into a vector where each element 
corresponds to the count of a specific word in the Text .

2.  Compute the cosine similarity between each pair of documents using the cosine_similarity. This will create a similarity matrix where the (i,j)-th element is the cosine similarity between user text  i and job offers j.

3. get recommendations for a given user, select the row of the similarity matrix that corresponds to the job , sort the similarities in descending order, and return the top k most similar jobs.




In [ ]:
all_dataframes=[]
for i in range(len(user)):
    # Select the row using iloc
    row = user.iloc[i]

    # Create a DataFrame for the row
    row_df = pd.DataFrame(row).T  # Transpose the row to convert it to a column
    row_df.columns = [f'{col}' for col in row_df.columns]  # Rename the columns to avoid duplicates
    user_count_segheweb = count_vect.transform([row_df['headline'].tolist()[0]])
    cos_sim_count_seghe = map(lambda x: cosine_similarity(user_count_segheweb, x), count_comb)
    rem1 = list(cos_sim_count_seghe)
    top10_seghe_count = sorted(range(len(rem1)), key = lambda i: rem1[i], reverse = True)[:10]
    list_scores_seghe_count = [rem1[i][0][0] for i in top10_seghe_count]
    df = get_recommendation(top10_seghe_count, comb_jobs_df, list_scores_seghe_count,[row_df['ApplicantID'].tolist()[0]])
    all_dataframes.append(df)
concatenated_df = pd.concat(all_dataframes, axis=0)
DFJOBSEEKRSRecommandation=concatenated_df
DFJOBSEEKRSRecommandation['Applicant_ID'] = DFJOBSEEKRSRecommandation['Applicant_ID'].apply(str).astype(str)
DFJOBSEEKRSRecommandation['Applicant_ID']= DFJOBSEEKRSRecommandation['Applicant_ID'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)


In [ ]:
DFJOBSEEKRSRecommandation.tail(20)

,Applicant_ID,jobID,CleanTitle,Score
0,783,2616,Business Intelligence Analyst,0.400892
1,783,2646,Business Intelligence Manager,0.395285
2,783,3224,Low Level Software Developer,0.392232
3,783,704,business intelligence and visualization specia...,0.381881
4,783,703,business intelligence developer,0.377964
5,783,2614,Business Intelligence Analyst,0.376889
6,783,103,Automotive Software Engineer,0.375
7,783,108,Automotive Software Engineer,0.375
8,783,254,Principal Software Architect,0.375
9,783,451,Software Engineer Sr Embedded C Orlando Fl,0.369274


## **Profile recommendation** 
To enhance the recommendation system by recommending each profile a job offer, you can use the same approach as before 

In [ ]:
def get_recommendationProfil(top, comb_jobs_all, scores,Profil):
  recommendation = pd.DataFrame(columns = ['Profil', 'jobID',  'CleanTitle', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Profil'] = Profil
      recommendation.at[count, 'jobID'] = comb_jobs_all['jobID'][i]
      recommendation.at[count, 'CleanTitle'] = comb_jobs_all['CleanTitle'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [ ]:
all_dataframesProfil=[]
for i in range(len(Profil)):
    # Select the row using iloc
    row = Profil.iloc[i]

    # Create a DataFrame for the row
    row_df = pd.DataFrame(row).T  # Transpose the row to convert it to a column
    row_df.columns = [f'{col}' for col in row_df.columns]  # Rename the columns to avoid duplicates
    user_count_segheweb = count_vect.transform([row_df['Text'].tolist()[0]])
    cos_sim_count_seghe = map(lambda x: cosine_similarity(user_count_segheweb, x), count_comb)
    rem1 = list(cos_sim_count_seghe)
    top10_seghe_count = sorted(range(len(rem1)), key = lambda i: rem1[i], reverse = True)[:10]
    list_scores_seghe_count = [rem1[i][0][0] for i in top10_seghe_count]
    df = get_recommendationProfil(top10_seghe_count, comb_jobs_df, list_scores_seghe_count,[row_df['Field'].tolist()[0]])
    all_dataframesProfil.append(df)
concatenated_dfPROFIL = pd.concat(all_dataframesProfil, axis=0)

In [ ]:
concatenated_dfPROFIL['Profil'] = concatenated_dfPROFIL['Profil'].apply(str).astype(str)
concatenated_dfPROFIL['Profil']= concatenated_dfPROFIL['Profil'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)
Profil.reset_index(drop=False, inplace=True)
#Profil=Profil.drop('Unnamed: 0',axis=1)
concatenated_dfPROFIL

,Profil,jobID,CleanTitle,Score
0,BI,726,business analyst,0.780074
1,BI,2595,Business Analyst Business Analyst,0.747789
2,BI,2616,Business Intelligence Analyst,0.734503
3,BI,1600,Business Data Analyst,0.727116
4,BI,1472,Data Analyst W,0.725815
...,...,...,...,...
5,gl,274,Entry Level Software Engineer,0.598561
6,gl,281,Entry Level Software Engineer,0.598561
7,gl,2118,Backend Software Engineer,0.595515
8,gl,386,software engineer,0.589423


## 👨‍💻 Find profile similarity
  To identify profiles that are similar to a new user based on a profile database, you can use a function that computes the cosine similarity between the new user's profile and all other profiles in the database. The cosine similarity score is a measure of the similarity between two vectors and can be used to identify profile that is the most similar to the new user's profile. Once the similarity scores are computed, the function can return the top profile that are most similar to the new user's profile. This can be useful for making personalized recommendations to the new user based on the profiles of other users in the database

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_user_profile(user_text, profile_data):
    # Create CountVectorizer object to convert text to vectors
    vectorizer = CountVectorizer()

    # Fit the vectorizer to the user text
    vectorizer.fit_transform([user_text])

    # Get the vector for the user text
    user_vector = vectorizer.transform([user_text]).toarray()[0]

    # Compute the cosine similarity between the user vector and each profile vector
    similarity_scores = []
    for    index , row  in profile_data.iterrows():
        profile_text =  row['Title']
        profile_vector = vectorizer.transform([profile_text]).toarray()[0]
        similarity_scores.append(cosine_similarity([user_vector, profile_vector])[0, 1])

    # Find the profile with the highest similarity score
    best_match_index = similarity_scores.index(max(similarity_scores))
    field_string = profile_data.loc[profile_data['index']==best_match_index, "Field"].to_string(index=False)
    field_string=field_string.replace("'","")
    

    return  field_string


In [ ]:
user1 = pd.read_csv('/content/jobseekersfinal.csv')
user1 = user1.rename(columns={'Unnamed: 0': 'ApplicantID'})
test_jobSeekers=user1.tail(30)
Text=test_jobSeekers.loc[test_jobSeekers['ApplicantID'] == 785, 'headline'].values
Job_Seeker_Test=test_jobSeekers.loc[test_jobSeekers['ApplicantID'] == 785]

In [ ]:
Text=test_jobSeekers.loc[test_jobSeekers['ApplicantID'] == 785, 'headline'].values
Porfile=find_user_profile(str(Text),Profil)
Porfile

'BI'

In [ ]:
Job_Seeker_Test

,ApplicantID,headline,fullName,allSkills,cluster
763,785,business intelligence | big data,ghassene ayari,"Microsoft Power BI, Project Management, Market...",Bi


#Job Recommendation System
 

##collaborative filtering recommender system
This technique recommends jobs to candidates based on the similarity between candidate profiles. It uses historical data on candidate behavior, such as job applications and job offers, to compute the similarity between candidate profiles and recommend jobs that are similar to the jobs that other candidates with similar profiles have applied to or accepted.

In [ ]:
#Users Applicant vector
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
applicant_vector = tf.fit_transform((user['Text'])) 

In [ ]:
#New User Score
jobS_record_vector = tf.transform(Job_Seeker_Test['headline'])
candidate_cos_similarity = map(lambda x: cosine_similarity(jobS_record_vector, x),applicant_vector)
candidate_score = list(candidate_cos_similarity)

In [ ]:
def get_candidates(top, df, scores):
  print(top)
  candidates = pd.DataFrame(columns = ['NewJobSeeks', 'ApplicantID', 'keywords', 'score'])
  count = 0
  for i in top:
    print(i)
    candidates.at[count, 'NewJobSeeks'] = 1
    candidates.at[count, 'ApplicantID'] = df[df['ApplicantID'==i]]
    candidates.at[count, 'score'] =  scores[count]
    count += 1
  return candidates

In [ ]:
top_candidates = sorted(range(len(candidate_score)), key=lambda i: candidate_score[i], reverse=True)[:5]
list_scores = [candidate_score[i][0][0] for i in top_candidates]
user_userwise = user.index.isin(top_candidates)
df1 = pd.DataFrame(data = user[user_userwise], columns=['ApplicantID','headline','allSkills'])
ApplicantList = pd.DataFrame(data = user[user_userwise], columns=['ApplicantID'])
ApplicantList=ApplicantList['ApplicantID'].to_list()

In [ ]:
df1

,ApplicantID,headline,allSkills
200,205,erp-business intelligence engineer (big data |...,"SQL Server Reporting Services (SSRS), SQL Serv..."
243,249,it engineer - data analyst,"Java, Symfony, PHP, Talend, Mathématiques, Dat..."
507,517,business intelligence | big data | data science.,"ETL Tools, Python (Programming Language), Micr..."
509,519,business intelligence | big data,"Microsoft Power BI, Project Management, Market..."
729,750,business intelligence | big data | data science.,"ETL Tools, Python (Programming Language), Micr..."


## Content-based filtering
 This technique recommends jobs to candidates based on the similarity between job descriptions and candidate profiles

In [ ]:
user_tfidf_gbade = count_vect.transform(Job_Seeker_Test['headline'])
cos_sim_tfidf_gbade = map(lambda x: cosine_similarity(user_tfidf_gbade, x), count_comb)

In [ ]:
rec1 = list(cos_sim_tfidf_gbade)
top10_seghe_tfidf = sorted(range(len(rec1)), key = lambda i: rec1[i], reverse = True)[:10]
list_scores_seghe_tfidf = [rec1[i][0][0] for i in top10_seghe_tfidf]
job_userwise = comb_jobs_df.index.isin(top10_seghe_tfidf)
dfcontent = pd.DataFrame(data = comb_jobs_df[job_userwise], columns=['jobID'])
ContenttList=dfcontent['jobID'].to_list()

In [ ]:
Job_list = comb_jobs_df['jobID'].isin(ContenttList) 
df_temp = pd.DataFrame(data = comb_jobs_df[Job_list], columns=['jobID','CleanTitle','Skills_Description','jobLocation'])
df_temp

,jobID,CleanTitle,Skills_Description,jobLocation
64,64,Regional Sales Manager,"marketing, business, monitoring, reporting, co...","Abuja, Territoire de la capitale fédérale du N..."
714,714,business intelligence analyst,"business intelligence, analytics, big data, da...",usa
781,781,Big Data Developer/Data Engineer,"big data, business, sql, hadoop\\big, computer...","London, ON"
2551,2551,Business Analyst Nantes,"cloud, data management, can, business, bi, big...","Nantes, Pays de la Loire, France"
2570,2570,Business Insights Manager,"machine learning, statistics, software enginee...","Quezon City, Région Capitale Nationale, Philip..."
2582,2582,Business Info Mgmt Analyst II,"support, database, design, material, nih, ad&d...","Toronto, ON"
2612,2612,Business Intelligence Consultant,"framework, machine learning, design, business,...","Londres, Angleterre, Royaume-Uni"
2613,2613,Business Analyst Gradu,"data visualization, business, marketing, suppo...","Wembley, Angleterre, Royaume-Uni"
2893,2893,Full time - Financial Data Analyst (Remote),"business, support, finance, accounting, microsoft","London, Ontario, Canada"


## Hybrid job recommendation system
now , we will combines multiple recommendation techniques, such as content-based filtering, collaborative filtering, and profile-based recommendation, to provide more accurate and diverse job recommendations

In [ ]:
def jobs_UserRecommendation(usrid_list):
    jobs_userwise = DFJOBSEEKRSRecommandation['Applicant_ID'].astype(int).isin(usrid_list)
    df1 = pd.DataFrame(data = DFJOBSEEKRSRecommandation[jobs_userwise], columns=['jobID'])
    joblist = df1['jobID'].tolist()
    return joblist

In [ ]:
def jobs_Profilecommendation(profil):
  jobs_userwise=concatenated_dfPROFIL[concatenated_dfPROFIL['Profil']==profil]['jobID'].tolist()
  return jobs_userwise

In [ ]:
ListUser=jobs_UserRecommendation(ApplicantList)
ListProfil=jobs_Profilecommendation(Porfile)
result_list = list(set(ListUser +  ListProfil + ContenttList))

In [ ]:
Job_list = comb_jobs_df['jobID'].isin(result_list) 
df_temp = pd.DataFrame(data = comb_jobs_df[Job_list], columns=['jobID','CleanTitle','Skills_Description','Country'])
DfHybrid = df_temp.drop_duplicates()
DfHybrid

,jobID,CleanTitle,Skills_Description,Country
64,64,Regional Sales Manager,"marketing, business, monitoring, reporting, co...",Nigéria
65,65,Software Engineer Frontend Applic,"can, react, cloud, typescript, mobile, ios, do...",Germany
710,710,associate business intelligence analyst,"reporting, finance, business intelligence, dat...",États-Unis
714,714,business intelligence analyst,"business intelligence, analytics, big data, da...",États-Unis
715,715,business intelligence analyst i,"bi, business, data analysis, databases, data v...",États-Unis
723,723,business analyst (ref: pr2090),"business, business process",États-Unis
726,726,business analyst,business,États-Unis
781,781,Big Data Developer/Data Engineer,"big data, business, sql, hadoop\\big, computer...",Canada
783,783,BIG DATA DEVELOPER/DATA ENGINEER,"big data, business, sql, hadoop\\big, computer...",Canada
1472,1472,Data Analyst W,"business, cloud",États-Unis


#Insufficient skills and course recommendation 

To assist interns and job seekers in improving their resumes and skill sets, we suggest providing them with a list of skills that are not currently included in their profile and of similar users. In addition to this, we can also recommend a selection of relevant online courses that will help them acquire new skills and improve their employability.

### Colaborative Skills and Profil Skills
 get the missing skills of the new jobseeker in comparison to other users

In [ ]:
SKILLS1 = Job_Seeker_Test['allSkills'].apply(lambda x: x.lower()).values
SKILLS1 = ', '.join(str(skill) for skill in SKILLS1)
ColaborativeSkill = ','.join(df1.groupby(level=0)['allSkills'].apply(lambda x: ','.join([f.lower() for f in x])).values)
ColaborativeSkill = ','.join(set(ColaborativeSkill.split(',')))
# split string1 into a list of elements
elements = ColaborativeSkill.split(',')
# extract elements from the list that are not in string2
result = [elem for elem in elements if elem not in SKILLS1]

 get the missing skills of the new jobseeker in comparison to his profil

In [ ]:
ProfilSkills = Profil[Profil['Field']==Porfile]['Technologies'].apply(lambda x: x.lower()).values
ProfilSkills = ', '.join(str(skill) for skill in ProfilSkills)
# split string1 into a list of elements
elementsProfil = ProfilSkills.split(',')
# extract elements from the list that are not in string2
result2 = [elem for elem in elementsProfil if elem not in SKILLS1]

In [ ]:
FinalList=result2+result

In [ ]:
print(FinalList)

['visualization', ' sci', ' artificial intelligence', ' erp', ' cryptocurrency', ' sap', ' search engines', ' ms excel', ' alteryx', ' aws lambda', ' mlops', ' business intelligence', ' ruby', ' devops', ' azure', ' ia', ' erp systems', ' hbase', ' airflow', ' postgresql', ' etl', ' blockchain', ' sql', ' business;support', ' mdm', ' data warehouse', ' snowflake', ' data modeling', ' statistics', ' powerbi', ' marketing', ' microsoft power', ' microsoft sql server', ' mysql', ' agile data services', ' accounting', ' django', ' data visualization', ' qlik', ' jenkins', ' talend', ' github', ' cdp', ' deep learning', ' tableautime series', ' oracle', ' data science', ' redis', ' exl', ' google tag manager', ' erp accounting', ' git', ' ai', ' mongodb', ' hadoop', ' scala', ' analytics', ' kubernetes', ' data acquisition', ' nosql', ' kos', ' docker', ' the african union commission', ' text mining', ' kafka', ' big data', ' first class', ' vba', ' tibco spotfire/explorer', ' information s

In [ ]:
!pip install skillNer
!python -m spacy download en_core_web_lg
# imports
import spacy
import en_core_web_lg
from spacy.matcher import PhraseMatcher

# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
from skillNer.general_params import SKILL_DB
# init params of skill extractor
nlp = en_core_web_lg.load()
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)


In [ ]:
result_string = ', '.join(FinalList)
annotations = skill_extractor.annotate(result_string)
skill_extractor.describe(annotations)

/usr/local/lib/python3.10/dist-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


In [ ]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform(doc)
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())
TOP_K_KEYWORDS = 20 # top k number of keywords to retrieve in a ranked document
vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True)
vectorizer.fit_transform(FinalList)
feature_names = vectorizer.get_feature_names_out()
SkillsFinal=get_keywords(vectorizer, feature_names,FinalList)

In [ ]:
SkillsFinal

['vmware',
 'visualization',
 'vba',
 'teamwork',
 'talend',
 'table',
 'symfony',
 'statistics',
 'sql',
 'spss',
 'snowflake',
 'shiny',
 'sci',
 'scala',
 'sap',
 'ruby',
 'redis',
 'qlik']

In [ ]:
# Tokenize the list of skills
skill_tokens = word_tokenize(' '.join(FinalList))
# Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in skill_tokens if token.lower() not in stop_words]
# Count the frequency of each skill
skills_counter = Counter(filtered_tokens)

# Get the top 10 most frequent skills
top_skills = skills_counter.most_common(20)
for skill, count in top_skills:
    print(skill)


data
sql
microsoft
server
services
learning
analytics
big
visualization
intelligence
erp
business
azure
power
accounting
talend
(
)
java
systems


### course recommendation 
After identifying the skills that are missing, we will recommend a list of online courses from Udemy and Coursera to each user. To determine the best courses to suggest, we used a similarity score based on the skills required by the courses and the skills that the user needs to acquire

In [ ]:
tfidf_vect = TfidfVectorizer()
# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((CoursData['course_title'])) 
tfidf_comb
def recommend_courses(df, skills):
    # Preprocess the course names to create a corpus of documents
    corpus = df['course_title'].apply(lambda x: ' '.join(x.lower().split()))

    # Convert the corpus into a matrix of TF-IDF features
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

    # Initialize an empty list to store recommended courses for each skill
    recommended_courses = []

    # Loop over the skills and calculate the cosine similarity with course names
    for skill in skills:
        skill_tfidf = tfidf_vectorizer.transform([' '.join(skill.lower().split())])
        cosine_similarities = cosine_similarity(skill_tfidf, tfidf_matrix).flatten()

        # Sort the courses based on their cosine similarity scores in descending order
        course_indices = cosine_similarities.argsort()[::-1]

        # Append the top 3 courses as recommendations for the current skill
        top_courses = df[['course_title', 'url','level']].iloc[course_indices[:3]]
        recommended_courses.append(top_courses)

    # Concatenate the recommended courses into a single dataframe
    recommended_courses_df = pd.concat(recommended_courses, ignore_index=True)
    recommended_courses_df = recommended_courses_df.drop_duplicates()
    return recommended_courses_df
recommended_courses = recommend_courses(CoursData,SkillsFinal)

In [ ]:
recommended_courses

,course_title,url,level
0,Networking and Security Architecture with VMwa...,https://www.coursera.org/learn/networking-secu...,Intermediate
1,Architecting with Google Kubernetes Engine: Pr...,https://www.coursera.org/learn/deploying-secur...,Intermediate
2,Instant Harmonica - play 'I just called to say...,https://www.udemy.com/instant-harmonica-play-i...,All Levels
3,Data Visualization,https://www.coursera.org/learn/datavisualization,Beginner
4,Python Data Visualization,https://www.coursera.org/learn/python-visualiz...,Advanced
5,Data Management and Visualization,https://www.coursera.org/learn/data-visualization,Advanced
6,"Excel/VBA for Creative Problem Solving, Part 2",https://www.coursera.org/learn/excel-vba-for-c...,Advanced
7,"Excel/VBA for Creative Problem Solving, Part 1",https://www.coursera.org/learn/excel-vba-for-c...,Beginner
8,"Excel/VBA for Creative Problem Solving, Part 3...",https://www.coursera.org/learn/excel-vba-for-c...,Advanced
9,Teamwork Skills: Communicating Effectively in ...,https://www.coursera.org/learn/teamwork-skills...,Intermediate
